In [ ]:
import glob 
import os
import sys

try:
    sys.path.append(glob.glob('../carla/dist/carla-*%d.%d-%s.egg' % (
        sys.version_info.major,
        sys.version_info.minor,
        'win-amd64' if os.name == 'nt' else 'linux-x86_64'))[0])
except IndexError:
    pass

import carla
import random
import time
import threading
import queue
import numpy as np

Connect to the server and initialize synchronous mode

In [ ]:
client = carla.Client('localhost', 2000)
client.set_timeout(2.0)

world = client.get_world()

In [ ]:
# Create a synchronous mode context
settings = world.get_settings()
settings.synchronous_mode = True
settings.fixed_delta_seconds = 0.1  # Simulation step size in seconds
world.apply_settings(settings)

In [ ]:
for actor in world.get_actors():
    if actor.type_id[:7] == "vehicle":
        ego_vehicle = actor

Spawn a Second vehicle and set it to autopilot mode

In [ ]:
bp = random.choice(world.get_blueprint_library().filter('*vehicle*'))
transform = random.choice(world.get_map().get_spawn_points())
second_vehicle = world.try_spawn_actor(bp, transform)
second_vehicle.set_autopilot(True)

Does imitating second vehicle's behavior work? No

In [ ]:
while True:
    ego_vehicle.apply_control(second_vehicle.get_control())

Does using `VehiclePIDController` class work? No

Reference: agents/navigation/controller.py

In [ ]:
from agents.navigation.controller import VehiclePIDController

args = dict(K_P=1, K_I=0, K_D=0)
controller = VehiclePIDController(ego_vehicle,args_lateral=args,args_longitudinal=args)
map = world.get_map()

In [ ]:
while True:
    waypoint = map.get_waypoint(second_vehicle.get_location(),project_to_road=True)

    speed = second_vehicle.get_velocity()
    target_speed = (speed.x**2 + speed.y**2 + speed.z**2)**0.5

    controller.run_step(waypoint=waypoint,target_speed=target_speed)

Does using *agents* work? **Yes**

Reference: https://carla.readthedocs.io/en/0.9.14/adv_agents/

In [ ]:
from agents.navigation.behavior_agent import BehaviorAgent

agent = BehaviorAgent(ego_vehicle, behavior='aggressive')

In [ ]:
agent.ignore_traffic_lights(True)
agent.set_destination(second_vehicle.get_location())
# Follow second vehicle
while True:
    if agent.done():
        agent.set_destination(second_vehicle.get_location())

    ego_vehicle.apply_control(agent.run_step())

To store desired data in seperate files in the *out* folder

In [ ]:
def process_data(queue_data, output_folder):
    while True:
        try:
            # Wait for 1 second for data to be available in the queue
            data = queue_data.get(timeout=1.0)  

            # Create a filename using the timestamp of the data
            timestamp = str(int(time.time() * 1000))  # Milliseconds since epoch
            filename = os.path.join(output_folder, f'data_{timestamp}.txt')

            # Write the data to the file
            with open(filename, 'w') as f:
                f.write(str(data))

        except queue.Empty:
            continue

In [ ]:
output_folder = '/home/parsa/Cheetah/carla_simulator/out'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [ ]:
# Set up the data processing thread
data_queue = queue.Queue()
data_thread = threading.Thread(target=process_data, args=(data_queue, output_folder))
data_thread.start()

In [ ]:
record_time = 60 # Seconds
time0 = time.time()
while True:  
    # Put the data into the queue
    data = {
        'vehicle_location': (ego_vehicle.get_location().x,ego_vehicle.get_location().y,ego_vehicle.get_location().z),
    }
    data_queue.put(data)

    time1 = time.time()
    if (time1-time0) > record_time:
        break

Attempt to draw boxes around vehicles

https://github.com/carla-simulator/carla/issues/1212

In [ ]:
crashed = False
while not crashed:
    world.wait_for_tick()
    for vehicle in world.get_actors().filter("vehicle*"):
        transform = vehicle.get_transform()
        bounding_box = vehicle.bounding_box
        bounding_box.location += transform.location
        world.debug.draw_box(bounding_box, transform.rotation)
        if vehicle != ego_vehicle:
            distance = ego_vehicle.get_location().distance(vehicle.get_location())
            world.debug.draw_string(vehicle.get_location(), f'{distance:.1f} m')

In [ ]:
crashed = False
while not crashed:
    for vehicle in world.get_actors().filter("vehicle*"):
        if vehicle != ego_vehicle:
            distance = ego_vehicle.get_location().distance(vehicle.get_location())
            if distance < 100:
                loc = vehicle.get_location()
                loc.z += 4
                world.debug.draw_string(loc, f'{distance:.1f} m')

Reference: https://carla.readthedocs.io/en/latest/tuto_G_bounding_boxes/

In [ ]:
import cv2

for actor in world.get_actors():
    if actor.type_id == "sensor.camera.rgb":
        camera = actor

# Create a queue to store and retrieve the sensor data
image_queue = queue.Queue()
camera.listen(image_queue.put)

In [ ]:
def get_image_point(loc, K, w2c):
        # Calculate 2D projection of 3D coordinate

        # Format the input coordinate (loc is a carla.Position object)
        point = np.array([loc.x, loc.y, loc.z, 1])
        # transform to camera coordinates
        point_camera = np.dot(w2c, point)

        # New we must change from UE4's coordinate system to an "standard"
        # (x, y ,z) -> (y, -z, x)
        # and we remove the fourth componebonent also
        point_camera = [point_camera[1], -point_camera[2], point_camera[0]]

        # now project 3D->2D using the camera matrix
        point_img = np.dot(K, point_camera)
        # normalize
        point_img[0] /= point_img[2]
        point_img[1] /= point_img[2]

        return point_img[0:2]

In [ ]:
while True:

    # Retrieve and reshape the image
    image = image_queue.get()

    img = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

    # Get the camera matrix 
    world_2_camera = np.array(camera.get_transform().get_inverse_matrix())

    for npc in world.get_actors().filter('*vehicle*'):

        # Filter out the ego vehicle
        if npc.id != vehicle.id:

            bb = npc.bounding_box
            dist = npc.get_transform().location.distance(vehicle.get_transform().location)

            # Filter for the vehicles within 50m
            if dist < 50:

            # Calculate the dot product between the forward vector
            # of the vehicle and the vector between the vehicle
            # and the other vehicle. We threshold this dot product
            # to limit to drawing bounding boxes IN FRONT OF THE CAMERA
                forward_vec = vehicle.get_transform().get_forward_vector()
                ray = npc.get_transform().location - vehicle.get_transform().location

                if forward_vec.dot(ray) > 1:
                    p1 = get_image_point(bb.location, K, world_2_camera)
                    verts = [v for v in bb.get_world_vertices(npc.get_transform())]
                    x_max = -10000
                    x_min = 10000
                    y_max = -10000
                    y_min = 10000

                    for vert in verts:
                        p = get_image_point(vert, K, world_2_camera)
                        # Find the rightmost vertex
                        if p[0] > x_max:
                            x_max = p[0]
                        # Find the leftmost vertex
                        if p[0] < x_min:
                            x_min = p[0]
                        # Find the highest vertex
                        if p[1] > y_max:
                            y_max = p[1]
                        # Find the lowest  vertex
                        if p[1] < y_min:
                            y_min = p[1]

                    cv2.line(img, (int(x_min),int(y_min)), (int(x_max),int(y_min)), (0,0,255, 255), 1)
                    cv2.line(img, (int(x_min),int(y_max)), (int(x_max),int(y_max)), (0,0,255, 255), 1)
                    cv2.line(img, (int(x_min),int(y_min)), (int(x_min),int(y_max)), (0,0,255, 255), 1)
                    cv2.line(img, (int(x_max),int(y_min)), (int(x_max),int(y_max)), (0,0,255, 255), 1)


    cv2.imshow('ImageWindowName',img)
    if cv2.waitKey(1) == ord('q'):
        break